In [1]:
import os
import tqdm
import fitsio
import healpy
import numpy as np
from scipy.special import erf
from astropy.table import table as aTable

import desitarget.io
from desitarget.sv3.sv3_targetmask import bgs_mask as sv3_bgs_mask


Bad key text.latex.preview in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95 

In [2]:
os.sys.path.append(os.getcwd().replace('/nb', '/bin'))
import svda as SVDA

In [3]:
ff = fitsio.read('/global/cfs/cdirs/desi/spectro/redux/fuji/zcatalog/ztile-sv3-bright-cumulative.fits')

# select only BGS targets
is_bgs = ((ff['COADD_FIBERSTATUS'] == 0) & 
          ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_BRIGHT']) != 0) & 
          (ff['SPECTYPE'] == 'GALAXY') & 
          (ff['Z'] > 0.) & (ff['Z'] < 0.6))
ff = ff[is_bgs]
print('%i objects' % ff.shape[0])

233831 objects


In [4]:
photsys = ff['PHOTSYS'].astype(str)
trans_g = SVDA.mwdust_transmission(ff['EBV'], 'g', photsys, match_legacy_surveys=False)
trans_r = SVDA.mwdust_transmission(ff['EBV'], 'r', photsys, match_legacy_surveys=False)
trans_z = SVDA.mwdust_transmission(ff['EBV'], 'z', photsys, match_legacy_surveys=False)

flux_g = ff['FLUX_G'] / trans_g
flux_r = ff['FLUX_R'] / trans_r
flux_z = ff['FLUX_Z'] / trans_z
fiberflux_r = ff['FIBERFLUX_R'] / trans_r

gmag = 22.5 - 2.5 * np.log10(flux_g.clip(1e-7))
rmag = 22.5 - 2.5 * np.log10(flux_r.clip(1e-7))
zmag = 22.5 - 2.5 * np.log10(flux_z.clip(1e-7))

rfib = 22.5 - 2.5 * np.log10(fiberflux_r.clip(1e-7))

In [5]:
target_ids = ff['TARGETID']

In [6]:
with open('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/altmtl/SV3HPList.txt') as f: 
    hplist = [int(hp) for hp in f.readline().split(',')]

In [ ]:
nobs = np.zeros((len(target_ids), 128))
for imtl in range(1):#28): 
    # compile mtls
    mtls = []
    for pix in hplist:
        targs = desitarget.io.read_mtl_ledger(
            os.path.join('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/altmtl/DESI_EDA_SV3AltMTLs/', 
                         'Univ%s' % str(imtl).zfill(3), 'sv3', 'bright', 'sv3mtl-bright-hp-%i.ecsv' % pix), 
            unique=True, isodate=None, initial=False, leq=False)
        mtls.append(targs)
    mtls = np.concatenate(mtls)
    mtls = mtls[mtls['NUMOBS'] > 0.5]
    
#     for i in tqdm.tqdm(range(len(target_ids))):                
#         if np.sum(mtls['TARGETID'] == target_ids[i]): 
#             nobs[i,imtl] = 1

 46%|████▌     | 107274/233831 [02:55<03:25, 615.99it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 24%|██▎       | 55127/233831 [01:18<04:03, 734.13it/s]

In [ ]:
nobs_mtls = np.sum(nobs, axis=1)

In [ ]:
# calculate the weights
w_mtls = 129. / (nobs_mtls + 1)